In [101]:
# MNIST
# DataLoader, Transformation
# Multilayer Nerual Net, activation function
# Loss and Optimizer
# Training Loop
# Model evaluation
# GPU support

In [102]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [103]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

input_size = 784 # 28x28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

cuda


In [104]:
train_datasets = torchvision.datasets.MNIST(root='./data', train=True,transform=transforms.ToTensor(), download=True)
test_datasets = torchvision.datasets.MNIST(root='./data', train=False,transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=False)

examples = iter(train_loader)
samples, labels = examples.next()
print(samples.shape, labels.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [105]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [106]:
#Loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [107]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        #forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        #backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) % 100 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1 / 10, step 100/600, loss = 1.8032
epoch 1 / 10, step 200/600, loss = 1.3684
epoch 1 / 10, step 300/600, loss = 0.9970
epoch 1 / 10, step 400/600, loss = 0.7456
epoch 1 / 10, step 500/600, loss = 0.7366
epoch 1 / 10, step 600/600, loss = 0.6256
epoch 2 / 10, step 100/600, loss = 0.4901
epoch 2 / 10, step 200/600, loss = 0.4624
epoch 2 / 10, step 300/600, loss = 0.4730
epoch 2 / 10, step 400/600, loss = 0.4445
epoch 2 / 10, step 500/600, loss = 0.3460
epoch 2 / 10, step 600/600, loss = 0.3461
epoch 3 / 10, step 100/600, loss = 0.3711
epoch 3 / 10, step 200/600, loss = 0.2287
epoch 3 / 10, step 300/600, loss = 0.2357
epoch 3 / 10, step 400/600, loss = 0.3882
epoch 3 / 10, step 500/600, loss = 0.3150
epoch 3 / 10, step 600/600, loss = 0.2431
epoch 4 / 10, step 100/600, loss = 0.3611
epoch 4 / 10, step 200/600, loss = 0.3673
epoch 4 / 10, step 300/600, loss = 0.2899
epoch 4 / 10, step 400/600, loss = 0.3298
epoch 4 / 10, step 500/600, loss = 0.3489
epoch 4 / 10, step 600/600, loss =

In [108]:
#test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predictions = torch.max(outputs,1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

acc = 100.0 *n_correct/n_samples
print(f'accuracy = {acc}')

accuracy = 94.0
